In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [32]:
# Chargement des données
tasks = pd.read_csv('Task_catagories.csv')
engineers = pd.read_csv('engineers_skills.csv')

In [33]:
# Prétraitement des compétences
tasks['Skill'] = tasks['Skill'].str.lower().str.strip()
engineers['Skills'] = engineers['Skills'].str.lower().str.split(',').apply(lambda x: {s.strip() for s in x})


In [40]:
# Configuration du modèle avec normalisation explicite
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_split=10,
    max_features=0.8,
    random_state=42
)


In [42]:
# Entraînement avec données préparées
feature_names = ['Skill Match', 'Years Experience', 'Current Workload', 'Performance Rating']

def train_model():
    # Création de données d'entraînement réalistes
    synthetic_data = []
    for _ in range(1000):
        skill_match = np.random.rand()
        experience = np.random.randint(0, 20)
        workload = np.random.randint(0, 40)
        rating = np.random.uniform(2.5, 5.0)

        target = (
            0.6 * skill_match +
            0.3 * (experience / 20) +
            0.1 * (rating / 5.0) +
            np.random.normal(0, 0.03)
        )

        synthetic_data.append([
            skill_match,
            experience,
            workload,
            rating,
            np.clip(target, 0, 1)
        ])

    df = pd.DataFrame(synthetic_data, columns=feature_names + ['Target'])
    model.fit(df[feature_names], df['Target'])

train_model()


In [48]:
def recommend_engineers(required_skills):
    required_skills = {s.strip().lower() for s in required_skills}

    results = []
    for _, eng in engineers.iterrows():
        eng_skills = eng['Skills']
        match = len(required_skills & eng_skills) / len(required_skills) if required_skills else 0

        # Création du DataFrame avec noms de colonnes explicites
        input_data = pd.DataFrame([[
            match,
            eng['Years Experience'],
            eng['Current Workload'],
            eng['Performance Rating']
        ]], columns=feature_names)

        score = model.predict(input_data)[0]
        results.append((
            eng['Engineer'],
            np.clip(score, 0, 1),  # Assurance d'un score entre 0 et 1
            eng['Years Experience'],
            eng['Performance Rating']
        ))

    # Tri multicritère amélioré
    return sorted(results,
                key=lambda x: (-x[1], -x[2], -x[3]))[:5]


In [55]:
# Interface utilisateur avec gestion d'erreurs
try:
    task_desc = input("Description de la tâche : ")
    skills = input("Compétences requises (séparées par virgules) : ").split(',')

    recommendations = recommend_engineers(skills)

    print(f"\nRecommandations pour '{task_desc}':")
    for i, (name, score, exp, rating) in enumerate(recommendations, 1):
        print(f"{i}. {name} (Score: {score:.2f}, Exp: {exp}ans")

except Exception as e:
    print(f"Erreur : {str(e)}")

Description de la tâche : Optimize server performance
Compétences requises (séparées par virgules) : typescript

Recommandations pour 'Optimize server performance':
1. Soumaya Ammar (Score: 0.86, Exp: 14ans
2. Nadia Ben Ali (Score: 0.85, Exp: 13ans
3. Ahmed Zarrouk (Score: 0.84, Exp: 13ans
4. Salma Chaabane (Score: 0.83, Exp: 12ans
5. Fatma Saïdi (Score: 0.80, Exp: 11ans
